In [1]:
import pandas as pd
df = pd.read_pickle("sample_data.pkl")
df = df[df['label'] > 0]
df['label'] = df['label']-1
len(set(df.label))

15

In [5]:
sample = pd.DataFrame()
for i in set(df.label):
    try:
        sample = sample.append(df[df['label'] == i].sample(5000))
    except:
        sample = sample.append(df[df['label'] == i])
df = sample

C:\Users\samue\AppData\Local\Temp\ipykernel_5172\853090621.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(df[df['label'] == i].sample(5000))
C:\Users\samue\AppData\Local\Temp\ipykernel_5172\853090621.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(df[df['label'] == i].sample(5000))
C:\Users\samue\AppData\Local\Temp\ipykernel_5172\853090621.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(df[df['label'] == i].sample(5000))
C:\Users\samue\AppData\Local\Temp\ipykernel_5172\853090621.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample = sample.append(df[

In [6]:
import numpy as np
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

48086 6011 6011


In [7]:
from WordToVector import *
train_embeddings = transformer(df_train)
val_embeddings = transformer(df_val)
test_embeddings = transformer(df_test)

In [8]:
from to_torch_dataset import *

batch_size = 32

train, val = Dataset_embedding(df_train, train_embeddings), Dataset_embedding(df_val, val_embeddings)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

In [11]:
import torch.nn as nn

class LSTM(nn.ModuleList):

	def __init__(self):
		super(LSTM, self).__init__()
		
		# Hyperparameters
		self.batch_size = 32
		self.hidden_dim = 50
		self.LSTM_layers = 2
		self.input_size = 512
		
		self.dropout = nn.Dropout(0.5)
		self.lstm = nn.LSTM(input_size=self.hidden_dim, hidden_size=self.hidden_dim, num_layers=self.LSTM_layers, batch_first=True)
		self.fc1 = nn.Linear(in_features=self.hidden_dim, out_features=self.hidden_dim*2)
		self.fc2 = nn.Linear(self.hidden_dim*2, 1)
		
	def forward(self, x):
		
		# Hidden and cell state definion
		h = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
		c = torch.zeros((self.LSTM_layers, x.size(0), self.hidden_dim))
		
		# Initialization fo hidden and cell states
		torch.nn.init.xavier_normal_(h)
		torch.nn.init.xavier_normal_(c)

		# Feed LSTMs
		out, (hidden, cell) = self.lstm(x, (h,c))
		out = self.dropout(out)
		# The last hidden state is taken
		out = torch.relu_(self.fc1(out[:,-1,:]))
		out = self.dropout(out)
		out = torch.sigmoid(self.fc2(out))

		return out


In [12]:
from torch.optim import Adam
from tqdm import tqdm

model = LSTM()

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-5)


In [ ]:
epochs = 5

if use_cuda:

        model = model.cuda()
        criterion = criterion.cuda()

for epoch_num in range(epochs):

        total_acc_train = 0
        total_loss_train = 0

        for train_input, train_label in tqdm(train_dataloader):

            train_label = train_label.type(torch.LongTensor)
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            
            acc = (output.argmax(dim=1) == train_label).sum().item()
            total_acc_train += acc

            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
        
        total_acc_val = 0
        total_loss_val = 0

        with torch.no_grad():

            for val_input, val_label in val_dataloader:

                val_label = val_label.type(torch.LongTensor)
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                
                acc = (output.argmax(dim=1) == val_label).sum().item()
                total_acc_val += acc
        
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
            | Train Accuracy: {total_acc_train / len(train_data): .3f} \
            | Val Loss: {total_loss_val / len(val_data): .3f} \
            | Val Accuracy: {total_acc_val / len(val_data): .3f}')



In [ ]:
import torch.optim

# Defines a RMSprop optimizer to update the parameters
optimizer = optim.RMSprop(self.model.parameters(), lr=args.learning_rate)

for epoch in range(args.epochs):

  predictions = []

  # model in training mode
  self.model.train()

  for x_batch, y_batch in self.loader_training:

    x = x_batch.type(torch.LongTensor)
    y = y_batch.type(torch.FloatTensor)

    # Feed the model and get output "y_pred"
    y_pred = self.model(x)

    # Calculate loss
    loss = F.binary_cross_entropy(y_pred, y)

    # The gradientes are calculated
    # i.e. derivates are calculated
    loss.backward()
    
    # Each parameter is updated
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    optimizer.step()
    
    # Take the gradients to zero!
    # a.grad.zero_()
    # b.grad.zero_()
    optimizer.zero_grad()

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda')
#embeddings = model.encode(list(df['text']), convert_to_tensor=True)
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [13]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
model = model.append(models.LSTM(768, 256))
model = model.append(models.Dense(in_features=768, out_features=2, activation_function=nn.Tanh()))
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): LSTM(
    (encoder): LSTM(768, 256, batch_first=True, bidirectional=True)
  )
  (3): Dense({'in_features': 768, 'out_features': 2, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [19]:
import pandas as pd
df = pd.read_pickle("sample_data.pkl")
df = df[df['label'] > 0]
df = df.dropna()
df = df[df['text'] != 'nan']
df['label'] = df['label'] - 1
df.head(5)

,text,label,tokenized_words
234,儘管李文亮支持香港警察、擁護黨，是港人所謂的「愛國藍絲」，但他的敢言還是值得尊重。大陸藍絲能...,0,儘 管 李 文 亮 支持 香港 警察 、 擁護黨 ， 是 港人 所謂 的 「 愛國 藍絲 」...
356,【鼠王FUN之影張相攞份獎】 肺炎疫情持續，令香港充斥咗唔少負能量...啱啱踏入鼠年，有線一...,0,【 鼠王 FUN 之影 張 相 攞 份 獎 】 肺炎 疫情 持續 ， 令 香港 充斥 咗...
726,《疫情之後的盛世》補充篇 昨日文章出街後，有黃絲說是自High文章，這一點我不怪對方，畢竟有...,0,《 疫情 之 後 的 盛世 》 補充 篇 昨日 文章 出街 後 ， 有黃絲 說 是 自 ...
898,2019今日在西灣河的警民衝突，警方進入聖十字架堂拘捕五人，其後教堂執事被群眾指罵及推撞。天...,0,2019 今日 在 西灣河 的 警民 衝突 ， 警方 進入 聖 十字架 堂 拘捕 五人 ， ...
977,【鼠王FUN之影張相攞份獎】 肺炎疫情持續，令香港充斥咗唔少負能量...啱啱踏入鼠年，有線一...,0,【 鼠王 FUN 之影 張 相 攞 份 獎 】 肺炎 疫情 持續 ， 令 香港 充斥 咗...


In [20]:
df = df.groupby('label', group_keys=False).apply(lambda x: x.sample(min(len(x), 5000)))

import numpy as np
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

47991 5999 5999


In [21]:
df_train.head(3)

,text,label,tokenized_words
196995,【7.23役情最前線】美限令中共關閉侯斯頓領館；中共怕美區分中共與中國；美駐華使館聲明：停止...,9,【 7.23 役情 最 前線 】 美 限令 中共 關閉 侯斯頓 領館 ； 中共 怕 美區 分...
248546,【疫情下嚴重影響學習進度 ● 教你化危為機小貼士】 (最尾有驚喜！) . 由於香港疫情仍未受...,4,【 疫情 下嚴 重影 響學習 進度 ● 教 你化 危為 機小 貼士 】 ( 最尾...
138812,【侵侵輸唔起？曝白宮向美國疾控中心施壓調低死亡病例數字】 據美媒Daily Beast當地時...,1,【 侵侵輸 唔 起 ？ 曝白宮 向 美國 疾控中心 施壓 調低 死亡 病例 數字 】 據...


In [23]:
from sentence_transformers.readers import InputExample
train_examples = []

for i in range(len(df_train.label)):
    sample = list(df[df['label'] == i].text) 
    train_examples.append(InputExample(texts=sample, label=i))
train_examples

 ...]

In [10]:
train_batch_size = 2

In [15]:
from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import InputExample

from torch.utils.data import DataLoader

train_examples = [InputExample(texts=['First pair, sent A', 'First pair, sent B'], label=0),
    InputExample(texts=['Second Pair, sent A', 'Second Pair, sent B'], label=1)]

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
train_loss = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=2)

In [16]:
model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


{'input_ids': tensor([[    0, 23972, 80836,     4,  9325,    62,     2,     1],
        [    0, 77648,  1342,   481,     4,  9325,    62,     2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0'), 'token_embeddings': tensor([[[ 6.9044e-02, -7.8245e-02, -1.3219e-02,  ...,  6.6546e-02,
           1.9568e-02, -4.5218e-02],
         [ 1.1700e-01, -1.4179e-01, -1.4210e-02,  ...,  1.0534e-01,
           1.0898e-01, -9.6148e-02],
         [-4.7845e-02,  6.2185e-02, -1.4974e-03,  ...,  1.8331e-01,
           8.7783e-02, -7.5565e-03],
         ...,
         [ 9.1254e-02, -2.3842e-01, -1.3403e-02,  ...,  5.1401e-02,
           5.6716e-02, -5.7823e-02],
         [ 1.0845e-01, -3.8738e-02, -1.2177e-02,  ...,  1.3216e-01,
           8.0352e-02, -6.1083e-02],
         [ 1.9363e-05, -1.3670e-01, -1.9083e-02,  ...,  8.5259e-02,
           6.4103e-02, -1.0177e-01]],

        [[ 6.3118e-02,  1.3298e-02, -1.0580e-02,  ...,  4

KeyError: 'sentence_lengths'